# Sparse Document Retriever - Implantação

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
from model_sparse_retriever import TfidfRetriever, W2VRetriever, BM25Retriever

In [2]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
import json


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/sparse_retriever.joblib")
        self.model = artifacts["model"]
        self.report_contexts = artifacts["report_contexts"]
    
    def build_result_dataframe(self, sim_contexts_ids,scores):
        sim_contexts = [self.report_contexts[i] for i in sim_contexts_ids[0]]
        df = pd.DataFrame({'doc_id':sim_contexts_ids[0],'score':scores[0],'sim_contexts':sim_contexts})
        df = df.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        return df

    def predict(self, X, feature_names, meta=None):

        question = X.copy()[0]
        sim_contexts_ids,scores = self.model(question)
        df = self.build_result_dataframe(sim_contexts_ids,scores)
        df['question'] = [question]*len(scores[0])
        #teste = json.dumps({'question':[question]*len(scores[0]),'df':df.to_numpy()})
        #*** TypeError: Object of type ndarray is not JSON serializable
        return df.to_numpy() 

Overwriting Model.py


In [3]:
from Model import Model
model = Model()
model.predict(["Qual é o melhor herbicida para erva da ninha ?"],None)

array([[162, 18.098672345272938,
        ' 1.3 Palavras-chave: triketonas, sulfoniluréia, fitotoxicidade, rendimento relativo',
        'Qual é o melhor herbicida para erva da ninha ?'],
       [115, 18.098672345272938,
        ' 1.3 Palavras-chave: triketonas, sulfoniluréia, fitotoxicidade, rendimento relativo',
        'Qual é o melhor herbicida para erva da ninha ?'],
       [689, 18.098672345272938,
        ' 1.3 Palavras-chave: triketonas, sulfoniluréia, fitotoxicidade, rendimento relativo',
        'Qual é o melhor herbicida para erva da ninha ?'],
       [4, 17.896872862638563,
        ' 1.3 Palavras-Chave: clomazone, sulfentrazone, cloransulam, s-metolachlor, produtividade',
        'Qual é o melhor herbicida para erva da ninha ?'],
       [39, 17.896872862638563,
        ' 1.3 Palavras-Chave: clomazone, sulfentrazone, cloransulam, s-metolachlor, produtividade',
        'Qual é o melhor herbicida para erva da ninha ?'],
       [309, 17.819838897351964,
        ' 1.3 Palavras-Ch